<a href="https://colab.research.google.com/github/ASO1801023/AI_teach2020/blob/master/2020AI0404_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Doc2VeCで文章を学習したコーパスモデルを作って保存

### データリストの作成

In [1]:

#学習対象とする青空文庫の作品リスト --- (*1)
list = [
    {"author":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
    ]},
    {"author":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"},
    ]},
    {"author":{
        "name":"ポー エドガー・アラン",
        "url":"https://www.aozora.gr.jp/cards/000094/files/"}, 
     "books":[
        {"name":"ウィリアム・ウィルスン","zipname":"2523_ruby_19896.zip"},
        {"name":"落穴と振子","zipname":"1871_ruby_17551.zip"},
        {"name":"黒猫","zipname":"530_ruby_20931.zip"},
        {"name":"群集の人","zipname":"56535_ruby_69925.zip"},
        {"name":"沈黙","zipname":"56537_ruby_70425.zip"},
    ]},
    {"author":{
        "name":"紫式部",
        "url":"https://www.aozora.gr.jp/cards/000052/files/"}, 
     "books":[
        {"name":"源氏物語 01 桐壺","zipname":"5016_ruby_9746.zip"},
        {"name":"源氏物語 02 帚木","zipname":"5017_ruby_9752.zip"},
        {"name":"源氏物語 03 空蝉","zipname":"5018_ruby_9754.zip"},
        {"name":"源氏物語 04 夕顔","zipname":"5019_ruby_9761.zip"},
        {"name":"源氏物語 05 若紫","zipname":"5020_ruby_11253.zip"},
    ]},
]

### MeCabのインストール

In [2]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!pip install mecab-python3 > /dev/null

# MeCabの実行時の指定パスをインストールパスにリンクさせる
# シンボリックリンク（/etc/mecabrcを/usr/local/etc/mecabrcで参照できるようにする）
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

### データリストを検索して順番に返す関数：book_listを定義

In [3]:
# 作品リスト(list)をループして順次リターンする
def book_list():
  for novellist in list:
    author = novellist["author"]
    for book in novellist["books"]:
      # yieldで順次値をリターンして次のループへ進む
      yield author, book

### 指定された書籍名（book）のzipファイルを開き、ファイルのテキストデータを読み込んでリターンする関数：read_book

In [4]:
import zipfile
import os.path
import urllib.request as req
# zipファイルを開き、中の文書データを取得する
def read_book(author, book):
  zipname = book["zipname"]
  # zipファイルが無ければダウンロードする
  if not os.path.exists(zipname):
    req.urlretrieve(author["url"] + zipname, zipname)
  zipname = book["zipname"]
  # zipファイルを開く
  with zipfile.ZipFile( zipname, "r" ) as zf:
    # Zipファイルに含まれるファイルを開く
    for filename in zf.namelist():
      # txt拡張子ファイル以外は処理をスキップ
      if os.path.splitext(filename)[1] != ".txt":
        continue
      with zf.open(filename, "r") as f:
        # 今回読み込むファイルはShift-JISなので文字コードを指定して解読する（デコード）
        return f.read().decode("shift-jis") # デコードした値をリターン

### MeCabのインスタンス生成と、引数で渡された文章データを形態素解析した配列にしてリターンする関数：split_words

In [5]:
import MeCab
# MeCabインスタンスの生成
mecab = MeCab.Tagger()

# 引数のテキストを形態素解析＆ストップワード除去して配列にしてリターン
def split_words(text):
  node = mecab.parseToNode(text)
  # 分かち書き（形態素解析＆ストップワード除去）して結果用配列
  wakati_words = []
  while node is not None:
    hinshi = node.feature.split(",")[0]
    if hinshi in ["名詞"]:
      wakati_words.append(node.surface)
    elif hinshi in ["動詞", "形容詞"]:
      wakati_words.append( node.feature.split(",")[6] )
    node = node.next
  # while終わり
  return wakati_words


### 学習モデル作成のメイン処理。作品リストをループしながらzipファイルを開き、テキストデータを形態素解析してタグ（ラベル）付きドキュメントにして、そのドキュメントを学習させる

In [ ]:
from gensim import models
from gensim.models.doc2vec import TaggedDocument
# 作品リストをDoc2Vecが学習できるTaggedDocument（説明変数＆目的変数）形式にし、配列に追加
documents = [] # 追加していく配列
# 作品リストをループして回す
for author, book in book_list():
  # 作品の文字列文章データを取得
  words = read_book( author, book )
  # 作品の文字列文章データを分かち書きにする
  wakati_words = split_words(words)
  # TaggedDocumentの作成　説明変数=分かち書きにした作品 タグ=作者：作品名
  document = TaggedDocument( wakati_words, [author["name"] +":"+book["name"]] )
  # 登録するタグドキュメントをデバッグ表示
  print(document)
  # 配列に登録
  documents.append(document)
# forループ終わり

# TaggedDocumentの配列を使ってDoc2Vecの学習モデルを作成
model = models.Doc2Vec( documents, dm=0, vector_size=300, window=15, min_count=1 )
# 作成した学習モデルをファイル保存
model.save('aozora.model')
print("モデル作成完了")



## Doc2Vecで文章を学習したコーパスモデルを使って、関連度の高い文章を推論

### 保存されたコーパス学習モデルファイルを読み込む

In [ ]:
from gensim import models
# 保存したDoc2Vec学習モデルファイルを読み込み
model = models.Doc2Vec.load('aozora.model')

### 指定されたzipファイルをダウンロードし、テキストデータを読み込んでリターンする関数：readbookを定義

In [ ]:
import urllib.request as req
import zipfile
import os.path

# 分類用のzipファイルを開き、中の文章データを取得する
def read_book( url, zipname ):
  # 指定されたzipファイルが存在しなければ、urlからダウンロード
  if not os.path.exists(zipname):
    req.urlretrieve( url, zipname )
  # 指定されたzipファイルを開いて、ファイルデータを読み込んでリターン
  with zipfile.ZipFile( zipname, "r" ) as zf:
    for filename in zf.namelist():
      with zf.open( filename, "r" ) as f:
        return f.read().decode("shift-jis")

### MeCabインスタンスの生成と、引き渡されたテキストを形態素解析してリターンする関数：split_words

In [ ]:
### モデル保存処理と同じものなので再定義しません

### 引き渡されたタイトルとurlを読込、Doc2Vecの学習モデルから関連度の高い登録データを推論させる関数：similar

In [ ]:
# 引数のタイトル、urlの作品を分類する(関連度の高いデータを推論する)
def similar( title, url ):
  zipname = url.split("/")[-1]

  words = read_book( url, zipname )
  wakati_words = split_words(words)
  # 文章ベクトルを取得
  vector = model.infer_vector( wakati_words )
  print( "--- [", title, "」と似た作品は？ ---" )
  # 推論結果を表示する
  print( model.docvecs.most_similar( [vector], topn=3 ) )
  print("")

### Doc2Vecのコーパス学習モデルによる推論のメイン処理

In [ ]:
# メイン実行処理

# 引き渡したデータ（第2引数のzipデータ）と関連度の高いデータを表示
similar("宮沢 賢治:よだかの星", 
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛", 
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ポー エドガー・アラン:マリー・ロジェエの怪事件", 
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部:源氏物語 06 末摘花", 
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")
